In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import glob
import PIL.Image as Image
import torch.utils.data as data
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm


In [3]:
im1_mean = [0.4042127201928427, 0.412576527722026, 0.42259604676925305, 0.43350413711051056, 0.443855690032959, 0.45140969585826785, 0.45334100553503703, 0.4468871853247354, 0.43043786530179756, 0.404718756014092, 0.37331931684981945, 0.3418279141576013,0.31582148226006085]
im2_mean = [0.41915108148831637, 0.42160734423110063, 0.423962258067611, 0.42601487354034223, 0.4275087523492551, 0.4281458628877129, 0.4276228290175893, 0.4257433505811504, 0.4224049007987228, 0.41766727063885856, 0.411706916417739, 0.4048067182780746,0.39733730345961626]
im3_mean = [0.44033469101140527, 0.4503952708250446, 0.46193364562084055, 0.4738397130390685, 0.48420565326379844, 0.49040465635549285, 0.48956380687421636, 0.47946231783341775, 0.45967343462004484, 0.4323601088249301, 0.4019184607310354, 0.37350198569667487,0.3512268327779888]

im1_std = [0.14732023241999434, 0.147809241675677, 0.14804508827283613, 0.1477574222750175, 0.1470682801149269, 0.146539035212084, 0.14675348492637386, 0.14775991989824996, 0.14848621292810835, 0.1465088569090591, 0.13925408874284476, 0.12590440973744257,0.10823487264730804]
im2_std = [0.12246606074712985, 0.12329658876446659, 0.1242021620079717, 0.12510900974078862, 0.12595538841389356, 0.12670439991891755, 0.12730158828399146, 0.12765988385551708, 0.12760264169433708, 0.1269505535408886, 0.12552824561659184, 0.12318955457930125,0.11984089703152391]
im3_std = [0.11690682253979083, 0.11736897240085645, 0.11786639327518504, 0.11859482672002022, 0.11987161521520828, 0.1221401983344635, 0.12578272075012656, 0.13025746011922526, 0.13357738663561808, 0.13312490121042017, 0.12725185198165137, 0.11619303606414197,0.10181700640611481]

calibrator1 = {}
calibrator2 = {}

for i in range(13):
    layer_idx = i
    def calibration1(layer_data,layer_idx):
        mask = (layer_data >= 0.1) & (layer_data <= 0.9) 
        scaled_x = np.copy(layer_data)  
        scaled_x[mask] = ((layer_data[mask] - im2_mean[layer_idx])/im2_std[layer_idx]) * im1_std[layer_idx] + im1_mean[layer_idx] 
        return scaled_x
    
    def calibration2(layer_data,layer_idx):
        mask = (layer_data >= 0.1) & (layer_data <= 0.9) 
        scaled_x = np.copy(layer_data)  
        scaled_x[mask] = ((layer_data[mask] - im3_mean[layer_idx])/im3_std[layer_idx]) * im1_std[layer_idx] + im1_mean[layer_idx] 
        return scaled_x    

    calibrator1[layer_idx] = calibration1
    calibrator2[layer_idx] = calibration2
    
    

In [4]:
import numba
import numpy as np
from PIL import Image
import torch
from matplotlib import pyplot as plt
from torch.utils.data import Dataset
from typing import Tuple, List
from torchvision.transforms import transforms
import cv2

class VesuviusTrainData(Dataset):
    _relative_sv_dir = "surface_volume"  # relative to the directory of train data

    def __init__(self,
                 dir_path: str = "train/1",
                 z_start: int = 0,
                 z_end: int = 64,
                 nucleus_shape: Tuple[int, int] = (16, 16),
                 hull_size: Tuple[int, int] = (64, 64),
                 compress_depth =  None,
                 give_indx = False,
                 calibrator = None,
                 boundarysize = None
                 ):
        self.dir_path = dir_path

        self.z_start = z_start
        self.z_end = z_end

        self.nucleus_shape = nucleus_shape
        self.hull_size = hull_size
        
        self.compress_depth = compress_depth
        self.give_indx = give_indx
        self.calibrator = calibrator
        self.boundarysize = boundarysize
        
        
        self._setup()

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, index: int):
        i, j = self._get_pixel_from_index(self.indices[index])
        nucleus_height, nucleus_width = self.nucleus_shape

        if self.give_indx == False:
            if self.calibrator != None:
                return (torch.from_numpy(
                    self.calibrate(self.images[
                    :,
                    i - self._left_hull + self._left_pad_image:
                    i + nucleus_width + self._right_hull + self._left_pad_image,
                    j - self._top_hull + self._top_pad_image:
                    j + nucleus_height + self._bottom_hull + self._top_pad_image
                     ])),
                    torch.from_numpy(
                    self.inklabels[i: i + nucleus_width,
                    j: j + nucleus_height]))
            else:
                return (torch.from_numpy(
                    self.images[
                    :,
                    i - self._left_hull + self._left_pad_image:
                    i + nucleus_width + self._right_hull + self._left_pad_image,
                    j - self._top_hull + self._top_pad_image:
                    j + nucleus_height + self._bottom_hull + self._top_pad_image
                     ]),
                    torch.from_numpy(
                    self.inklabels[i: i + nucleus_width,
                    j: j + nucleus_height])) 
        else:
            if self.calibrator != None:
                return (torch.from_numpy(
                    self.calibrate(self.images[
                    :,
                    i - self._left_hull + self._left_pad_image:
                    i + nucleus_width + self._right_hull + self._left_pad_image,
                    j - self._top_hull + self._top_pad_image:
                    j + nucleus_height + self._bottom_hull + self._top_pad_image
                     ])),
                    torch.from_numpy(
                    self.inklabels[i: i + nucleus_width,
                    j: j + nucleus_height]),
                   np.stack(([i+k for k in range(nucleus_width)],[
                    j+k for k in range(nucleus_height)]),axis=1))
            else:
                return (torch.from_numpy(
                    self.images[
                    :,
                    i - self._left_hull + self._left_pad_image:
                    i + nucleus_width + self._right_hull + self._left_pad_image,
                    j - self._top_hull + self._top_pad_image:
                    j + nucleus_height + self._bottom_hull + self._top_pad_image
                     ]),
                    torch.from_numpy(
                    self.inklabels[i: i + nucleus_width,
                    j: j + nucleus_height]),
                   np.stack(([i+k for k in range(nucleus_width)],[
                    j+k for k in range(nucleus_height)]),axis=1)) 
            
    def calibrate(self,arr):
        calibrated_arr = np.zeros_like(arr)

        for i in range(arr.shape[0]):
            calibration_func = self.calibrator[i]
            calibrated_arr[i, :, :] = calibration_func(arr[i, :, :],i)
        return calibrated_arr


    def _load_images(self, z_start=None,z_end=None):
        if z_end is None:
            z_end = self.z_end 
        if z_start is None:
            z_start = self.z_start
            
        num_images = z_end - z_start
        self.images = np.empty(
            (
                num_images,
                self.mask.shape[0] + self._left_pad_image + self._right_pad_image,
                self.mask.shape[1] + self._top_pad_image + self._bottom_pad_image
            ), 
            dtype=np.float16
        )
        for index, i in enumerate(range(z_start, z_end)):
            # noinspection PyTypeChecker
            image = np.array(Image.open(f"{self.dir_path}"
                                        f"/{self._relative_sv_dir}"
                                        f"/{i:02d}.tif"),
                             dtype=np.float32) / 65535.0
            
            image = image.astype(np.float16)
            
            image = np.pad(image, ((self._left_pad_mask + self._left_pad_image,
                                    self._right_pad_mask + self._right_pad_image),
                                   (self._top_pad_mask + self._top_pad_image,
                                    self._bottom_pad_mask + self._bottom_pad_image)),
                           'constant', constant_values=0)
            self.images[index, :, :] = image
            
    def _load_scale(self):
        current_stack_layer = self.z_start
        images = np.empty(
            (
                len(self.compress_depth),
                self.mask.shape[0] + self._left_pad_image + self._right_pad_image,
                self.mask.shape[1] + self._top_pad_image + self._bottom_pad_image
            ), 
            dtype=np.float16
        )
        for index,layer in enumerate(self.compress_depth):
            self._load_images(current_stack_layer, current_stack_layer + layer)
            current_stack_layer += layer
            layer_stack = self.images
            images[index,:,:] = np.mean(layer_stack,axis = 0)
            
        self.images = images
        
        
    def crop_boundary(self,mask, boundary_size):
        mask = mask.astype(np.uint8)
        mask_binary = cv2.threshold(mask, 0.5, 1, cv2.THRESH_BINARY)[1]

        contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        boundary_mask = np.ones_like(mask, dtype=np.uint8)

        cv2.drawContours(boundary_mask, contours, -1, (0), 100)

        kernel_size = int(boundary_size)  # Adjust the kernel size based on your needs
        kernel = np.ones((kernel_size, kernel_size), dtype=np.uint8)
        eroded_boundary_mask = cv2.erode(boundary_mask, kernel, iterations=1)

        cropped_mask = mask * eroded_boundary_mask[:, :]

        return cropped_mask
    
    # Internal utility methods
    def _setup(self):
        # noinspection PyTypeChecker
        self.mask = np.array(Image.open(f"{self.dir_path}/mask.png"))
        if (self.boundarysize) != None:
            self.mask = self.crop_boundary(self.mask,self.boundarysize)
        # noinspection PyTypeChecker
        self.inklabels = np.array(Image.open(f"{self.dir_path}/inklabels.png"))
        self._granulate_mask()
        self._fill_new_mask()
        if self.compress_depth is None:
            self._load_images()
        else:
            self._load_scale()
        self._get_leftover_hull()

    def _granulate_mask(self):
        self._get_paddings()
        self.mask = np.pad(self.mask,
                           ((self._left_pad_mask, self._right_pad_mask),
                            (self._top_pad_mask, self._bottom_pad_mask)),
                           'constant', constant_values=0)
        self.indices = _find_labeled_nucleus(self.mask, self.nucleus_shape)
        self.inklabels = np.pad(self.inklabels, 
                                ((self._left_pad_mask, self._right_pad_mask),
                                 (self._top_pad_mask, self._bottom_pad_mask)),
                                'constant', 
                                constant_values=0)

    def _get_paddings(self):
        width, height = self.mask.shape
        nucleus_width, nucleus_height = self.nucleus_shape
        hull_width, hull_height = self.hull_size
        self._left_pad_mask = (width % nucleus_width) // 2
        self._right_pad_mask = width % nucleus_width - self._left_pad_mask
        self._top_pad_mask = (height % nucleus_height) // 2
        self._bottom_pad_mask = height % nucleus_height - self._top_pad_mask

        self._left_pad_image = (hull_width - nucleus_width) // 2
        self._right_pad_image = hull_width - nucleus_width - self._left_pad_image
        self._top_pad_image = (hull_height - nucleus_height) // 2
        self._bottom_pad_image = hull_height - nucleus_height - self._top_pad_image

    def _fill_new_mask(self):
        nucleus_width, nucleus_height = self.nucleus_shape
        self.mask = np.zeros_like(self.mask)
        for index in self.indices:
            i, j = self._get_pixel_from_index(index)
            self.mask[
                i * nucleus_width: (i + 1) * nucleus_width,
                j * nucleus_height: (j + 1) * nucleus_height
            ] = 1

    def _get_pixel_from_index(self, index: int):
        mask_width, mask_height = self.mask.shape
        nucleus_width, nucleus_height = self.nucleus_shape
        height = mask_height // nucleus_height
        return index // height * nucleus_height, index % height * nucleus_width

    def _get_leftover_hull(self):
        nucleus_width, nucleus_height = self.nucleus_shape
        hull_width, hull_height = self.hull_size
        self._left_hull = (hull_width - nucleus_width) // 2
        self._right_hull = hull_width - nucleus_width - self._left_hull
        self._top_hull = (hull_height - nucleus_height) // 2
        self._bottom_hull = hull_height - nucleus_height - self._top_hull


@numba.njit()
def _find_labeled_nucleus(mask, nuclues_shape):
    width, height = mask.shape
    nucleus_width, nucleus_height = nuclues_shape

    indices = - np.ones((width // nucleus_width) * (height // nucleus_height), dtype=np.int32)
    current_index = 0
    for i in range(width // nucleus_width):
        for j in range(height // nucleus_height):
            if np.sum(
                    mask[i * nucleus_width: (i + 1) * nucleus_width,
                         j * nucleus_height: (j + 1) * nucleus_height]
                      ) > 0:
                indices[current_index] = i * (height // nucleus_height) + j
                current_index += 1
    return indices[indices != -1]

In [5]:
nucleus_shape = (8,8)
hull_size = (24,24)
z_start = 22
z_end = 35
boundarysize = 700

# Training

In [6]:
data1 = VesuviusTrainData(
    dir_path="/kaggle/input/vesuvius-challenge-ink-detection/train/1"\
    ,nucleus_shape = nucleus_shape,hull_size=hull_size, z_start=z_start
    , z_end=z_end,give_indx=True,boundarysize = boundarysize+200)

In [42]:
from torch.utils.data import DataLoader
import cv2
import torch.nn.functional as F
example_loader = DataLoader(data1,batch_size=1,shuffle=True)

def zoom_3d_image(image, zoom_factor):
    # Calculate the target shape based on the zoom factor
    target_shape = (int(image.shape[0] * zoom_factor),
                    int(image.shape[1] * zoom_factor),
                    int(image.shape[2] * zoom_factor))
    
    # Perform trilinear interpolation to resize the image
    zoomed_image = F.interpolate(image.unsqueeze(0).unsqueeze(0), size=target_shape, mode='trilinear')
    
    # Upsample the zoomed image to the original size
    upsampled_image = F.interpolate(zoomed_image, size=image.shape, mode='nearest')
    
    # Remove the extra dimensions and return the upsampled image as a PyTorch tensor
    upsampled_image = upsampled_image.squeeze()
    
    return upsampled_image

def zoom_2d_image(image, zoom_factor):
    # Calculate the target shape based on the zoom factor
    target_shape = (int(image.shape[0] * zoom_factor),
                    int(image.shape[1] * zoom_factor))
    
    # Perform bilinear interpolation to resize the image
    zoomed_image = F.interpolate(image.unsqueeze(0).unsqueeze(0), size=target_shape, mode='bilinear')
    
    # Upsample the zoomed image to the original size
    upsampled_image = F.interpolate(zoomed_image, size=image.shape, mode='nearest')
    
    # Remove the extra dimensions and return the upsampled image as a PyTorch tensor
    upsampled_image = upsampled_image.squeeze()
    
    return upsampled_image

def crop_3d_image(image, crop_size):
    # Generate random starting coordinates within the valid range
    start_x = torch.randint(0, image.shape[0] - crop_size[0] + 1, (1,))
    start_y = torch.randint(0, image.shape[1] - crop_size[1] + 1, (1,))
    start_z = torch.randint(0, image.shape[2] - crop_size[2] + 1, (1,))
    
    # Calculate the end coordinates based on the starting coordinates and crop size
    end_x = start_x + crop_size[0]
    end_y = start_y + crop_size[1]
    end_z = start_z + crop_size[2]
    
    # Crop the image based on the computed coordinates
    cropped_image = image[start_x:end_x, start_y:end_y, start_z:end_z]
    
    # Upsample the cropped image to the original size
    upsampled_image = torch.nn.functional.interpolate(cropped_image.unsqueeze(0).unsqueeze(0), size=image.shape, mode='nearest')
    
    # Remove the extra dimensions and return the upsampled image as a PyTorch tensor
    upsampled_image = upsampled_image.squeeze()
    return upsampled_image

def crop_2d_image(image, crop_size):
    # Generate random starting coordinates within the valid range
    start_x = torch.randint(0, image.shape[0] - crop_size[0] + 1, (1,))
    start_y = torch.randint(0, image.shape[1] - crop_size[1] + 1, (1,))
    
    # Calculate the end coordinates based on the starting coordinates and crop size
    end_x = start_x + crop_size[0]
    end_y = start_y + crop_size[1]
    
    # Crop the image based on the computed coordinates
    cropped_image = image[start_x:end_x, start_y:end_y]
    
    # Upsample the cropped image to the original size
    upsampled_image = torch.nn.functional.interpolate(cropped_image.unsqueeze(0).unsqueeze(0), size=image.shape,mode='nearest')
    
    # Remove the extra dimensions and return the upsampled image as a PyTorch tensor
    upsampled_image = upsampled_image.squeeze()
    
    return upsampled_image
    

for i, (data, target, idx) in enumerate(example_loader):
    if torch.sum(target) > 10 and torch.sum(target) < 30:
        print(data)
        data = data.to(torch.float)
        print(data.shape)
        print(crop_3d_image(data[0],[12,22,22]))

        break
    

tensor([[[[0.3381, 0.3279, 0.3877,  ..., 0.3184, 0.3354, 0.3530],
          [0.2739, 0.2803, 0.2922,  ..., 0.3340, 0.3518, 0.3623],
          [0.3027, 0.3049, 0.2749,  ..., 0.3276, 0.3447, 0.3604],
          ...,
          [0.4141, 0.4165, 0.4214,  ..., 0.3733, 0.3816, 0.3967],
          [0.3960, 0.3936, 0.3916,  ..., 0.3467, 0.3645, 0.3833],
          [0.3176, 0.3047, 0.2932,  ..., 0.3916, 0.4148, 0.4348]],

         [[0.3684, 0.3623, 0.3982,  ..., 0.3347, 0.3513, 0.3826],
          [0.3020, 0.2908, 0.3276,  ..., 0.3425, 0.3623, 0.3745],
          [0.2844, 0.2737, 0.2803,  ..., 0.3354, 0.3491, 0.3625],
          ...,
          [0.4026, 0.3997, 0.3984,  ..., 0.3823, 0.3872, 0.3992],
          [0.3931, 0.3840, 0.3735,  ..., 0.3486, 0.3665, 0.3845],
          [0.3418, 0.3250, 0.3079,  ..., 0.3899, 0.4104, 0.4277]],

         [[0.4111, 0.4087, 0.4263,  ..., 0.3728, 0.3857, 0.4111],
          [0.3442, 0.3330, 0.3689,  ..., 0.3704, 0.3857, 0.3979],
          [0.2900, 0.2759, 0.3020,  ..., 0

In [40]:
data2 = VesuviusTrainData(
    dir_path="/kaggle/input/vesuvius-challenge-ink-detection/train/2"\
    ,nucleus_shape = nucleus_shape,hull_size=hull_size, z_start=z_start
    , z_end=z_end,give_indx=True,boundarysize = boundarysize+500,calibrator=calibrator1)

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (140973980 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [41]:
from torch.utils.data import ConcatDataset
merged_dataset = ConcatDataset([data1, data2])

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import random

class RandomNoise(object):
    def __init__(self, mean=0.0, std=1.0):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        noise = torch.randn_like(tensor) * self.std + self.mean
        return tensor + torch.abs(noise)
    
def rotate_image(data,target):
    degrees = np.random.choice([90, 180, 270])
    rotated_data = np.rot90(data, k=degrees // 90, axes= (-2, -1)).copy()
    rotated_target = np.rot90(target, k=degrees // 90, axes=(-2, -1)).copy()
    return torch.tensor(rotated_data),torch.tensor(rotated_target)

def random_shift(data,target, max_shift=4):
    x_shift = np.random.randint(-max_shift, max_shift)
    y_shift = np.random.randint(-max_shift, max_shift)
    shifted_data = torch.roll(data, shifts=(x_shift, y_shift), dims=(-2, -1))
    shifted_target = torch.roll(target, shifts=(x_shift, y_shift), dims=(-2, -1))
    
    return shifted_data,shifted_target

class CNN3D(nn.Module):
    def __init__(self):
        super(CNN3D, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc = nn.Linear(576  , 64)
        
        self.saved_loss = []
        self.saved_epoch = []
        
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool3d(x, kernel_size=2)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool3d(x, kernel_size=2)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool3d(x, kernel_size=2)
        x = x.view(x.size(0), -1)  
        x = self.fc(x)
        return x
    
    def augment(self, data, target):
        batch_size = data.size(0)
        rands = torch.rand(batch_size)

        for i in range(batch_size):
            if rands[i] < 0.1:
                data[i] = crop_3d_image(data[i],[13,22,22])
                target[i] = crop_2d_image(target[i],[7,7])
            elif rands[i] < 0.25:
                zoomf = random.uniform(0.7, 0.95)
                data[i] = zoom_3d_image(data[i],zoomf)
                target[i] = zoom_2d_image(target[i],zoomf)
            elif rands[i] < 0.35:
                data[i], target[i] = rotate_image(data[i],target[i])

                
        return data, target


    def fit(self, train_data,optimizer, criterion, num_epochs,verbose=False): 
        train_loader = DataLoader(train_data, batch_size = 512, shuffle=True)
        for epoch in range(num_epochs):
            loss_list = []
            f1_list = []
            for batch_idx, (data, target, idxs) in tqdm(enumerate(train_loader)):
                data = data.to(torch.float)
                data, target = self.augment(data,target)
                inputs = data.view(data.shape[0],1, *data.shape[1:])
                inputs = inputs.to(device)
                output = (self(inputs))
                target = target.to(device)
                target = target.to(torch.float)
                optimizer.zero_grad()
                loss = criterion(output, target.view(target.shape[0], -1))
                l2_lambda = 0.0005 
                l2_regularization = torch.tensor(0.)
                l2_regularization =  l2_regularization.to(device)
                for param in model.parameters():
                    l2_regularization += torch.norm(param, p=2)
                    loss += l2_lambda * l2_regularization
                
                loss_list.append(loss.item())
                loss.backward()
                optimizer.step()
                if batch_idx % 100 == 0:
                    print(np.mean(loss_list))
                
            if verbose == True:
                epoch_loss = np.mean(loss_list)
                self.save_loss(epoch_loss,epoch)
                print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")
            
                
    def save_loss(self,epoch_loss,epoch):
        self.saved_loss.append(epoch_loss)
        if len(self.saved_epoch) != 0:
            if self.saved_epoch[-1] > epoch:
                self.saved_epoch.append(self.saved_epoch[-1]+1)
            else:
                 self.saved_epoch.append(epoch)
        else:
            self.saved_epoch.append(epoch)

In [44]:
class HybridLoss(nn.Module):
    def __init__(self, weight_dice=0.5, weight_bce=0.5):
        super(HybridLoss, self).__init__()
        self.weight_dice = weight_dice
        self.weight_bce = weight_bce
        self.bce_loss = nn.BCEWithLogitsLoss()
        self.sigmoid = nn.Sigmoid()

    def forward(self, predicted, target):
        predicted_sigmoid = self.sigmoid(predicted)
        dice_loss = self.dice_loss(predicted_sigmoid, target)
        bce_loss = self.bce_loss(predicted, target)
        hybrid_loss = (self.weight_dice * dice_loss) + (self.weight_bce * bce_loss)
        return hybrid_loss

    def dice_loss(self, predicted, target):
        H = 8
        W = 8
        predicted = predicted.view(-1, H, W)  
        target = target.view(-1, H, W) 
        smooth = 1e-5
        intersection = torch.sum(predicted * target)
        union = torch.sum(predicted) + torch.sum(target)
        dice_coefficient = (2 * intersection + smooth) / (union + smooth)
        dice_loss = 1 - dice_coefficient
        return dice_loss

In [45]:
model = CNN3D()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.001,lr=0.005)
criterion = HybridLoss()
model.load_state_dict(torch.load("/kaggle/input/3dweights/new3d-2.pth", map_location=torch.device('cpu')))
model.to(device)

CNN3D(
  (conv1): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv2): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (fc): Linear(in_features=576, out_features=64, bias=True)
)

In [46]:
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.001,lr=0.001)

In [47]:
num_epochs = 1
model.fit(merged_dataset,optimizer,criterion,num_epochs,verbose = True)
#torch.save(model.state_dict(), 'new_3dweights.pth')

1it [00:03,  3.88s/it]

0.8888023495674133


101it [04:45,  2.89s/it]

0.841376024307591


201it [09:55,  2.82s/it]

...

0.7713779329233265


801it [39:00,  2.91s/it]

0.7637837669822607


801it [39:01,  2.92s/it]


KeyboardInterrupt: 

In [48]:
torch.save(model.state_dict(), 'modelnewaugments.pth')

# Validation

In [49]:
data3 = VesuviusTrainData(
    dir_path="/kaggle/input/vesuvius-challenge-ink-detection/train/3"\
    ,nucleus_shape = nucleus_shape,hull_size=hull_size, z_start=z_start
    , z_end=z_end,give_indx=True,calibrator=calibrator2)

In [50]:
score = []
preds = []
targs = []
val_loader =DataLoader(data3, batch_size = 512)
for data, target,idxs in tqdm((val_loader)):
    data = data.to(torch.float)
    output = model(data.view(data.shape[0],1,*data.shape[1:]).to(device))
    target = target.to(device).view(target.shape[0],64)
    target = target.to(output.dtype)
    score.append(nn.BCEWithLogitsLoss(pos_weight=torch.tensor(3))(output, target).item())
    preds.append((nn.Sigmoid()(output)).detach().cpu().numpy())
    targs.append(target.detach().cpu().numpy())

print(np.mean(score))

100%|██████████| 769/769 [19:55<00:00,  1.55s/it]

0.758097517707413


In [51]:
class Fscore:
    def __init__(self):
        self.epsilon = 1e-7
        self.tp = 0
        self.fp = 0
        self.fn = 0
    
    def accumulate(self, y_true, y_pred):
        # Flatten the input arrays to 1D
        y_true = np.ravel(y_true)
        y_pred = np.ravel(y_pred)
        
        # Update true positives, false positives, and false negatives
        self.tp += np.sum((y_true == 1) & (y_pred == 1))
        self.fp += np.sum((y_true == 0) & (y_pred == 1))
        self.fn += np.sum((y_true == 1) & (y_pred == 0))
    
    def get_score(self, beta=1):
       # Calculate precision, recall, and F-score
        precision = self.tp / (self.tp + self.fp + self.epsilon)
        recall = self.tp / (self.tp + self.fn + self.epsilon)
        f_score = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall + self.epsilon)
    
        return f_score

In [54]:
import math
import numpy as np

def round_elements(arr, threshold):
    rounded_arr = []
    for elem in tqdm(arr):
        rounded_elem = np.zeros_like(elem)
        for i in range(elem.shape[0]):
            for j in range(elem.shape[1]):
                if elem[i, j] < threshold:
                    rounded_elem[i, j] = math.floor(elem[i, j])  # Round down
                else:
                    rounded_elem[i, j] = math.ceil(elem[i, j])   # Round up
        rounded_arr.append(rounded_elem)
    return rounded_arr

In [61]:
threshold = 0.6

rounded_numbers = round_elements(preds,threshold)

100%|██████████| 769/769 [01:51<00:00,  6.89it/s]


In [62]:
score = Fscore()
for p,t in zip(rounded_numbers,targs):
     score.accumulate(t,p)
        
print(score.get_score(beta=0.5))

0.270818906390613


In [ ]:
(0.76,27.4)

In [ ]:
class PictureBuilder:
    def __init__(self,snippet_width=16,snippet_height=16):
        self.full_width = 6452
        self.full_height = 8298
    
        self.snippet_width = snippet_width
        self.snippet_height = snippet_height

        self.full_picture = np.zeros((self.full_height, self.full_width))
        

    def __call__(self,indx_matrix,snippet_matrix):
        height = indx_matrix[0]
        width = indx_matrix[1]
        temp = insert_snippet(self.full_picture,height,width,self.snippet_height\
                       ,self.snippet_width,snippet_matrix)
        self.full_picture = temp
        
    
    def _get(self):
        return self.full_picture

@numba.njit()
def insert_snippet(full_picture,height,width,sheight,swidth,smatrix):
    for i in range(sheight):
        for j in range(swidth):
            full_picture[height[i],width[j]] = smatrix[i,j]
    return full_picture

In [ ]:
picture = PictureBuilder(snippet_width=8,snippet_height=8)
pics_loader = DataLoader(data1, batch_size=1)
for batch_idx, (x_data, target, indx_arr) in tqdm(enumerate(pics_loader)):
    x_data = x_data.to(torch.float)
    x_data = x_data.view(1,*x_data.shape)
    x_data = torch.tensor((x_data).to(device))
    pred =  torch.sigmoid(model(x_data))
    pred_cpu = pred.detach().cpu().numpy()
    picture(indx_arr[0, :].T.numpy(), pred_cpu.reshape(8, 8))
    if batch_idx > 400000:
        break
        

In [ ]:
plt.imshow(np.round(picture._get()))

In [ ]:
picture2 = PictureBuilder(snippet_width=8,snippet_height=8)
pics_loader = DataLoader(data2, batch_size=1)
for batch_idx, (x_data, target, indx_arr) in tqdm(enumerate(pics_loader)):
    x_data = x_data.to(torch.float)
    x_data = x_data.view(1,*x_data.shape)
    x_data = torch.tensor((x_data).to(device))
    pred =  torch.sigmoid(model(x_data))
    pred_cpu = pred.detach().cpu().numpy()
    picture2(indx_arr[0, :].T.numpy(), pred_cpu.reshape(8, 8))
    if batch_idx > 400000:
        break

In [ ]:
plt.imshow(np.round(picture2._get()))

In [ ]:
import numpy as np
from PIL import Image
import imageio

# Define threshold range
threshold_range = np.arange(0.05, 0.7, 0.02)

# Create empty image sequence
images = []

# Iterate over each threshold
for threshold in threshold_range:

    # Apply rounding based on the threshold
    rounded_pixels = np.round(picture._get() < threshold)

    # Create a grayscale image from the rounded pixels
    image = Image.fromarray((rounded_pixels*255).astype(np.uint8), mode='L')

    # Append image to the sequence
    images.append(image)

# Save the sequence as a GIF
imageio.mimsave('grid_rounding.gif', images, duration=0.01)